<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/binance_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/binance_market.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# 必要ライブラリのインストール
! pip install --upgrade rbot
! pip install json2html
! pip install plotly
! pip install nbformat

In [2]:
# 必要ライブラリーのインポート
from json2html import *
from IPython.display import HTML

In [3]:
# rbotのBinanceMarketクラスのインポート
# BinanceMarketクラスは、BinanceのAPIを利用して、取引所の情報を取得するクラス
# BinanceConfigクラスは、BinanceMarketクラスのコンストラクタに渡す設定クラス
from rbot import BinanceConfig, BinanceMarket

In [4]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
config = BinanceConfig.BTCUSDT
HTML(json2html.convert(config.__str__()))

exchange_name,BN
trade_category,SPOT
trade_symbol,BTCUSDT
home_currency,USDT
foreign_currency,BTC
testnet,False
rest_endpoint,https://api.binance.com
public_ws_endpoint,wss://stream.binance.com:9443/ws
private_ws_endpoint,wss://stream.binance.com:9443/ws
history_web_base,https://data.binance.vision/data/spot/daily/trades
new_order_path,/api/v3/order


In [5]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
market = BinanceMarket(config)

market

start,end
1700352000000000,1700438399999000
"""2023-11-19T00:00:00.000000""","""2023-11-19T23:59:59.999000"""
days=,0


In [6]:
# 過去１日分のデータをダウンロード。ダウンロードしたデータは、marketオブジェクトの属性に格納される。
# Trueを指定すると、再ダウンロードを行う。Falseの場合はローカルに保存されている場合は、再ダウンロードを行わない。
market.download(1, False)

0

In [7]:
 # OHLCVを計算する。
# start_time: 開始時刻(UNIX時間[us]) 0の場合はDBにある最初のレコードから
# end_time: 終了時刻(UNIX時間[us])　 0の場合はDBにある最後のレコードまで
# window_sec: OHLCVを計算する時間幅(秒)

ohlcv = market.ohlcv(start_time=0, end_time=0, window_sec=60)

In [8]:
# ohlcvはpolarsのDataFrameオブジェクトとして格納されている。
ohlcv

time_stamp,open,high,low,close,vol,count
datetime[μs],f64,f64,f64,f64,f64,u32
2023-11-19 00:00:00,36568.11,36573.44,36568.1,36569.74,11.74684,643
2023-11-19 00:01:00,36569.75,36569.75,36558.0,36558.0,12.62352,686
2023-11-19 00:02:00,36558.01,36558.01,36546.53,36546.53,11.81049,605
2023-11-19 00:03:00,36546.54,36546.54,36540.0,36540.0,4.48524,559
2023-11-19 00:04:00,36540.0,36540.01,36540.0,36540.0,4.78616,269
2023-11-19 00:05:00,36540.01,36540.01,36527.65,36527.66,10.54882,505
2023-11-19 00:06:00,36527.66,36527.66,36527.65,36527.66,3.19942,414
2023-11-19 00:07:00,36527.66,36527.66,36511.65,36513.99,15.68434,851
2023-11-19 00:08:00,36514.0,36523.82,36513.99,36523.81,8.93118,529


In [9]:
# OHLCVの表示
import plotly.graph_objects as go

go.Figure(
    data=[
        go.Candlestick(
            x=ohlcv['time_stamp'],
            open=ohlcv['open'],
            high=ohlcv['high'],
            low=ohlcv['low'],
            close=ohlcv['close']
        )
    ]
).show()


In [10]:
# VAP(Volume At Price)を計算する。
# from_time: 開始時刻(UNIX時間[us]) 0の場合はDBにある最初のレコードから
# 戻り値はprice_unit毎に集計されたVAPのPolars DataFrameオブジェクト
vap = market.vap(start_time=0, end_time=0, price_unit=10)

vap

order_side,price,size
bool,f64,f64
true,36380.0,14.37649
false,36380.0,26.00824
false,36390.0,68.70698
true,36390.0,77.9526
false,36400.0,211.52373
true,36400.0,115.13995
true,36410.0,116.47809
false,36410.0,93.42196
true,36420.0,184.71307


In [11]:
go.Figure(
    data=[
        go.Scatter(
            x=vap['size'],
            y=vap['price']
        )
    ]
).show()